## Connect to Meilisearch

In [1]:
import meilisearch

meilisearch_client = meilisearch.Client("http://localhost:7700", "aStrongMasterKey")
print("Successfully connected to Meilisearch!")

Successfully connected to Meilisearch!


## Configure the embedder

In [2]:
index_name = "test"
index = meilisearch_client.index(index_name)
task = index.update_embedders(
    {"my_embedder": {"source": "userProvided", "dimensions": 3072}}
)
meilisearch_client.wait_for_task(task.task_uid)
print("Embedders have been updated successfully!")

Embedders have been updated successfully!


In [3]:
from google import genai
from dotenv import load_dotenv
from typing import Optional, List

load_dotenv()


def get_query_embedding(query_text: str) -> Optional[List[float]]:
    print(f"Generating embedding for query: '{query_text}'")
    try:
        google_client = genai.Client()
        result = google_client.models.embed_content(
            model="gemini-embedding-001", contents=query_text
        )
        if result is None:
            raise ValueError("No result returned from embedding model.")

        if not hasattr(result, "embeddings") or result.embeddings is None:
            raise ValueError("No embeddings found in result.")

        embeddings = result.embeddings[0].values
        return embeddings
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None

In [6]:
query = "Test content 2"
query_vector = get_query_embedding(query)

if query_vector:
    search_results = index.search(
        query,
        {
            "vector": query_vector,
            "hybrid": {"embedder": "my_embedder", "semanticRatio": 0.5},
            "showRankingScore": True,
        },
    )

    print("\n--- Semantic Search Results ---")
    for hit in search_results["hits"]:
        score = hit["_rankingScore"]
        print(f"- {hit['title']} - {hit['content']} - Score: {score:.4f}")

Generating embedding for query: 'Test content 2'

--- Semantic Search Results ---
- Collapse search results in Elasticsearch - Test content 2 - Score: 1.0000
- Pre-filtering with kNN search in Elasticsearch - Test content 3 - Score: 0.9462
- Change the heap size for Elasticsearch - Test content 1 - Score: 0.9424
